In [1]:
import json
from rouge import Rouge
import os
import pandas as pd
import numpy as np
import copy
import jieba
import sys
import re
import logging
import gensim
from preprocess_data import *
from gensim.models.word2vec import LineSentence
from sklearn import model_selection
from config import config_base
import vocab
config = config_base.config

In [2]:
df = organize_data(config.train_data)

In [3]:
df = deal_data(df)

In [4]:
df = df.sample(5000)

In [46]:
df = shorten_content_all(df, config.max_len)

KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt


shorten content, accuracy: 0.9618
max length: 300
min length: 6
mean length:180
median length:176


In [50]:
df_tmp= build_answer_range(df)

answer generation accuracy: 0.9595



In [37]:
sum(mm)

19

In [41]:
max(1, 0)

1

In [49]:
def build_answer_range(df):
    sys.setrecursionlimit(1000000)
    rouge = Rouge(metrics=['rouge-l'])

    def match(merge, answer, question):
        merge_list = jieba.lcut(merge)
        merge_len = len(merge_list)
        answer_list = jieba.lcut(answer)
        answer_len = len(answer_list)
        question_str = ' '.join(jieba.lcut(question))
        start = []
        end = []
        for i in range(0, merge_len-answer_len+1):
            if merge_list[i: i+answer_len] == answer_list:
                start.append(i)
                end.append(i+answer_len-1)
        if len(start) == 0:
            return -1, -1
        elif len(start) == 1:
            return start[0], end[0]
        else:
            scores = []
            # 前后扩展5个词
            for s, e in zip(start, end):
                s = max(s-5, 0)
                answer_can = ' '.join(merge_list[s: e+5])
                score = rouge.get_scores(answer_can, question_str, avg=True)['rouge-l']['r']
                scores.append(score)
            max_idx = np.argmax(scores)
            return start[max_idx], end[max_idx]

    merges = df[df['is_in']]['merge'].values
    answers = df[df['is_in']]['answer'].values
    questions = df[df['is_in']]['question'].values
    answer_range = [match(m, a, q) for m, a, q in zip(merges, answers, questions)]

    start, end = list(zip(*answer_range))
    df.loc[df['is_in'], 'answer_start'] = start
    df.loc[df['is_in'], 'answer_end'] = end

    merge_len = len(merges)
    right_len = (df['answer_start'] > -1).sum()
    print('answer generation accuracy: %.4f\n' % (right_len/merge_len))

    return df

In [31]:
190/5000

0.038